(sec-dask-gpu)=
# GPU

GPU 以及其他异构计算加速器被广泛用来加速深度学习，Dask 社区联合 NVIDIA，提供了基于 GPU 的数据科学工具包，以加速各类任务。

## Dask GPU 集群

[Dask-CUDA](https://docs.rapids.ai/api/dask-cuda/stable/) 对 `dask.distributed` 扩展，可以识别和管理 GPU 设备。

使用这些 GPU 设备前，先通过 `pip install dask_cuda` 安装 Dask-CUDA。跟 {numref}`sec-dask-distributed` 提到的`dask.distributed` 类似，Dask-CUDA 提供了一个单机的 `LocalCUDACluster`，`LocalCUDACluster` 会自动发现并注册该计算节点上的多张 GPU 卡，每张 GPU 自动配比一定数量的 CPU 核心。比如，我的环境有 4 张 GPU 卡，启动一个单机 Dask 集群，会自动启动 4 个 Dask Worker，每个 Worker 一张 GPU 卡。

In [1]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster()
client = Client(cluster)
client

/fs/fast/u20200002/envs/dispy/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37111 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:37111/status,
Dashboard: http://127.0.0.1:37111/status,Workers: 4
Total threads: 4,Total memory: 90.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46657,Workers: 4
Dashboard: http://127.0.0.1:37111/status,Total threads: 4
Started: Just now,Total memory: 90.00 GiB
Comm: tcp://127.0.0.1:36681,Total threads: 1
Dashboard: http://127.0.0.1:38373/status,Memory: 22.50 GiB
Nanny: tcp://127.0.0.1:41031,


我们也可以启动一个 Dask GPU 集群，先启动 Dask Scheduler：

```
dask scheduler
```

再在每个 GPU 节点上启动支持 GPU 的 Worker，构成一个 Dask GPU 集群。

```
dask cuda worker tcp://scheduler:8786
```

In [4]:
client = Client("10.0.0.3:8786")
client

<Client: 'tcp://10.0.0.3:8786' processes=8 threads=8, memory=180.00 GiB>

:::{note}
Dask-CUDA 只发现并注册这些 GPU，但无法做到 GPU 的隔离，其他非 Dask 的任务仍然可以抢占该 GPU。GPU 资源的隔离应该需要借助 Kubernetes 等容器技术。 
:::

## GPU 任务

并不是所有任务都能被 GPU 加速，GPU 主要加速一些计算密集型任务，比如机器学习和深度学习等。目前，Dask 支持的 GPU 上框架包括：

* [CuPy](https://cupy.dev/) 的 Dask 分布式版本
* [Dask-cuDF](https://docs.rapids.ai/api/dask-cudf/stable/) 将 GPU 加速版的 cuDF 横向扩展到 GPU 集群上

:::{note}
使用 NVIDIA 的 GPU 的各类库，应该将 CUDA 目录添加到 `PATH` 和 `LD_LIBRARY_PATH` 环境变量中，CuPy 和 cuDF 需要依赖所安装的动态链接库。
:::

### 案例：奇异值分解

下面的代码在 GPU 上进行奇异值分解，是一种适合 GPU 加速的任务。设置 `dask.config.set({"array.backend": "cupy"})` 即可将 Dask Array 的执行后端改为 GPU 上的 CuPy。

```python
import cupy
import dask
import dask.array as da

dask.config.set({"array.backend": "cupy"})
rs = dask.array.random.RandomState(RandomState=cupy.random.RandomState)
x = rs.random((10000, 1000), chunks=(1000, 1000))
u, s, v = dask.array.linalg.svd(x)
```